<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [104]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

from typing import Tuple
from glob import glob
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 25)
import pickle
from scipy.sparse import csr_matrix
import plotly.express as px
%matplotlib inline
from matplotlib import pyplot as plt

from datetime import datetime, timedelta

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [31]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'championship_data'

In [32]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [33]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [34]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [35]:
train_df[[f'time{i}' for i in range(1, 11)]] = train_df[[f'time{i}' for i in range(1, 11)]].apply(pd.to_datetime)
test_df[[f'time{i}' for i in range(1, 11)]] = test_df[[f'time{i}' for i in range(1, 11)]].apply(pd.to_datetime)

In [39]:
train_df['session_timespan'] = (train_df['time10'] - train_df['time1']).apply(lambda x: x.total_seconds())

train_df['unique_sites'] = train_df.nunique(axis=1)
train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
train_df['day_of_week'] = train_df['time1'].apply(lambda x: x.weekday())
train_df['end_hour'] = train_df['time10'].apply(lambda x: x.hour)
train_df['start_month'] = train_df['time1'].apply(lambda x: x.month)
train_df['start_year'] = train_df['time1'].apply(lambda x: x.year)
train_df['site_time'] = round(train_df['session_timespan'] / train_df['unique_sites'], 2)

In [40]:
test_df['session_timespan'] = (test_df['time10'] - test_df['time1']).apply(lambda x: x.total_seconds())

test_df['unique_sites'] = test_df.nunique(axis=1)
test_df['start_hour'] = test_df['time1'].apply(lambda x: x.hour)
test_df['day_of_week'] = test_df['time1'].apply(lambda x: x.weekday())
test_df['end_hour'] = test_df['time10'].apply(lambda x: x.hour)
test_df['start_month'] = test_df['time1'].apply(lambda x: x.month)
test_df['start_year'] = test_df['time1'].apply(lambda x: x.year)
test_df['site_time'] = round(test_df['session_timespan'] / test_df['unique_sites'], 2)

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [41]:
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя

Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут.

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [42]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   site1             253561 non-null  int64         
 1   time1             253561 non-null  datetime64[ns]
 2   site2             253561 non-null  float64       
 3   time2             253561 non-null  object        
 4   site3             253561 non-null  float64       
 5   time3             253561 non-null  object        
 6   site4             253561 non-null  float64       
 7   time4             253561 non-null  object        
 8   site5             253561 non-null  float64       
 9   time5             253561 non-null  object        
 10  site6             253561 non-null  float64       
 11  time6             253561 non-null  object        
 12  site7             253561 non-null  float64       
 13  time7             253561 non-null  object        
 14  site

In [43]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,...,site9,time9,site10,time10,session_timespan,unique_sites,start_hour,day_of_week,end_hour,start_month,start_year,site_time
session_id,,,,,,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,...,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00,7.0,14,11,5,11.0,10,2014,0.50
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,...,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53,85.0,12,11,3,11.0,7,2014,7.08
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,...,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36,84.0,13,15,4,15.0,12,2014,6.46
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,...,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23,4.0,13,10,1,10.0,11,2014,0.31
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,...,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44,13.0,15,15,4,15.0,5,2014,0.87


In [44]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   site1             82797 non-null  int64         
 1   time1             82797 non-null  datetime64[ns]
 2   site2             82797 non-null  float64       
 3   time2             82797 non-null  object        
 4   site3             82797 non-null  float64       
 5   time3             82797 non-null  object        
 6   site4             82797 non-null  float64       
 7   time4             82797 non-null  object        
 8   site5             82797 non-null  float64       
 9   time5             82797 non-null  object        
 10  site6             82797 non-null  float64       
 11  time6             82797 non-null  object        
 12  site7             82797 non-null  float64       
 13  time7             82797 non-null  object        
 14  site8             8279

**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [45]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [86]:
train_df['combined'] = train_df[[f'site{i}' for i in range(1, 11)]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_df['combined'] = test_df[[f'site{i}' for i in range(1, 11)]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [87]:
test = test_df[['session_timespan', 'combined',
                                                                             'unique_sites',
                                                                             'start_hour', 'day_of_week', 'end_hour', 'start_month', 'start_year', 'site_time']]

In [88]:
train, y = train_df.drop(['target'], axis=1)[['session_timespan', 'combined',
                                              'unique_sites',
                                                                                   'start_hour', 'day_of_week', 'end_hour', 'start_month', 'start_year', 'site_time']], train_df['target']

In [140]:
sgd = SGDClassifier(loss='log', random_state=17, class_weight='balanced')

In [141]:
def tokenize_text(text):
    text = text[0].split()
    return text

text_transformer = make_pipeline(
    ColumnTransformer([
        ("selector", "passthrough", ['combined'])
    ], remainder="drop"),
     CountVectorizer(stop_words=['0'], tokenizer=tokenize_text, lowercase=False))


In [142]:
non_text_features = make_pipeline(
    ColumnTransformer([
        ("selector", "passthrough", ['session_timespan', 'unique_sites', 'start_hour',
                                     'day_of_week', 'end_hour', 'start_month', 'start_year', 'site_time'])
    ], remainder="drop"),
     StandardScaler())

In [143]:
pipeline = Pipeline([
    ('text_features', FeatureUnion([
        ('vect', text_transformer),    # Extract counts from names.
        ('num_words', non_text_features),    # non text features.
    ])),
    ('clf' , sgd),     # Feed the output through a classifier.
])

**Сделайте кросс-валидацию**

In [144]:
sorted(metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [145]:
cross_val_score(pipeline, train, y, scoring='roc_auc')

array([0.97167545, 0.97595795, 0.97345866, 0.97461824, 0.97271223])

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [150]:
%%time
pipeline.fit(train, y)

prediction = pipeline.predict_proba(test)

Wall time: 4.21 s


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [155]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [156]:
write_to_submission_file(prediction[:, 1], os.path.join(PATH_TO_DATA, 'submit.csv'))